In [1]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Get List of Reports Available

In [2]:
import os
from glob import glob
import src.ExportProvider.IBRK.Extract as ibrk
from lxml import etree
import src.ReportingStrategies.Slovenia.ReportGeneration as sir
import src.ReportingStrategies.Slovenia.Mappers.IBRK as ibrkMapper
import src.ConfigurationProvider.Configuration as conf
from arrow import Arrow
import arrow

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "imports")
generatedDirectory = os.path.join(rootOfProject, "exports")
configDirectory = os.path.join(rootOfProject, "config")

configProvider = conf.ConfigurationProvider(configDirectory)
taxPayerInfo = configProvider.getConfig()

brokerExports = glob("*.xml", root_dir=reportsDirectory)
print(brokerExports)


def loadFileAndExtractLines(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    
    with open(fileLocation) as fobj:
        xml = fobj.read()

    root = etree.fromstring(xml)
    transactions = ibrk.extractFromXML(root)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibrk.mergeTrades(brokerReports)




reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2023"), toDate=arrow.get("2024"), ReportType=sir.EDavkiDocumentWorkflowType.SELF_REPORT)

dividendReport = sir.EDavkiDividendReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = dividendReport.createReportEnvelope()

convertedCommonFormat = ibrkMapper.getGenericDividendLineFromIBRKCashTransactions(mergedReports.cashTransactions)

csvReport = dividendReport.generateDataFrameReport(convertedCommonFormat)
csvReport.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)

xmlReport = dividendReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)



['Tax_Relevant_Export_2023.xml', 'Tax_Relevant_Export_2022.xml']


### Stock Trades

In [10]:



def loadFileAndExtractLines(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    
    with open(fileLocation) as fobj:
        xml = fobj.read()

    root = etree.fromstring(xml)
    transactions = ibrk.extractFromXML(root)
    return transactions

reportconfig = sir.EDavkiReportConfig(fromDate=arrow.get("2023"), toDate=arrow.get("2024"), ReportType=sir.EDavkiDocumentWorkflowType.SELF_REPORT)

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibrk.mergeTrades(brokerReports)

convertedCommonFormat = ibrkMapper.getGenericTradeLinesFromIBRKTrades(mergedReports)

# print(convertedCommonFormat)
tradeReport = sir.EDavkiTradesReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)
envelope = tradeReport.createReportEnvelope()


tradeCsv = tradeReport.generateDataFrameReport(convertedCommonFormat)
tradeCsv.to_csv(os.path.join(generatedDirectory, 'export-trades.csv'), index=False)



xmlReport = tradeReport.generateXmlReport(convertedCommonFormat, envelope)


etree.ElementTree(xmlReport).write(os.path.join(generatedDirectory, "Doh_KDVP_9.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


US22052L1044
DE0005190003
DE000A0LAUP1
FR0010208488
FR0000120271
DE000WAF3001
CA0156581070
US6742152076
MHY1146L1258
US91851C2017
US2908462037
MHY2685T1313
US40434L1052
US30303M1027
US7475251036
US7982411057
MHY8900D1085



ValueError: Could not find matching sell